In [1]:
import pymysql

In [2]:
import pymysql.cursors
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='34006503'
                            )

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

In [28]:
print(connection)

In [29]:
cur.execute("CREATE DATABASE ZomatoDB")

1

In [3]:
cur.execute('SHOW DATABASES')
for db in cur:
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testdb',)
('ZomatoDB',)


In [4]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='34006503',
                            database='ZomatoDB')

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

### Our database designed to store information in RESTAURANTS, CUISINES, EVALUATION, and RATING table.

In [53]:
# Create main table:
make_main_table = """CREATE TABLE main_table(
                        Restaurant_ID INT NOT NULL,
                        Restaurant_Name VARCHAR(255),
                        Country_Code INT,
                        City VARCHAR(255),
                        Address VARCHAR(255),
                        Cuisine VARCHAR(255),
                        Average_Cost_for_two INT,
                        Currency VARCHAR(255),
                        Table_booking BOOL,
                        Online_delivery BOOL,
                        Now_delivering BOOL,
                        Switch_menu BOOL,
                        Price_range INT,
                        Aggregate_rating FLOAT,
                        Rating_color VARCHAR(255),
                        Rating_text VARCHAR(255),
                        Votes VARCHAR(255),
                        Average_cost_USD INT,
                        Coordinates POINT NOT NULL,
                        PRIMARY KEY(Restaurant_ID));"""
                            
cur.execute(make_main_table)
connection.commit()

In [34]:
# Restaurants table will contain the data that describes the each restaurant 
make_restaurant_table = """CREATE TABLE restaurants(
                            Restaurant_ID INT NOT NULL,
                            Restaurant_Name VARCHAR(255),
                            Country_Code INT,
                            City VARCHAR(255),
                            Address VARCHAR(255),
                            Coordinates POINT NOT NULL,
                            Cuisine VARCHAR(255),
                            Average_Cost_for_two INT,
                            Currency VARCHAR(255),
                            Table_booking BOOL,
                            Online_delivery BOOL,
                            Aggregate_rating FLOAT,
                            PRIMARY KEY(Restaurant_ID));"""

cur.execute(make_restaurant_table)
connection.commit()

In [35]:
make_cuisines_table = """CREATE TABLE cuisines(
                            Cuisine_type VARCHAR(255),
                            Restaurant_ID INT,
                            Coordinates POINT,
                            PRIMARY KEY (Restaurant_ID),
                            FOREIGN KEY (Restaurant_ID)
                                REFERENCES restaurants(Restaurant_ID));
                            """
cur.execute(make_cuisines_table)
connection.commit()

In [36]:
make_evaluation_table = """CREATE TABLE evaluation(
                                Table_booking BOOL,
                                Oneline_delivery BOOL,
                                Now_delivering BOOL,
                                Switch_menu BOOL,
                                Price_range INT,
                                Restaurant_ID INT,
                                PRIMARY KEY (Restaurant_ID),
                                FOREIGN KEY (Restaurant_ID)
                                    REFERENCES restaurants(Restaurant_ID)
                                );"""
cur.execute(make_evaluation_table)
connection.commit()

In [37]:
# correct rating -> Aggregate rating
make_rating_table = """CREATE TABLE ratings(
                            Rating FLOAT, 
                            Rating_color VARCHAR(255),
                            Rating_text VARCHAR(255),
                            Votes VARCHAR(255),
                            Restaurant_ID INT,
                            PRIMARY KEY (Restaurant_ID),
                            FOREIGN KEY (Restaurant_ID)
                                REFERENCES restaurants(Restaurant_ID));"""

cur.execute(make_rating_table)
connection.commit()

In [54]:
cur.execute('SHOW TABLES')
for tb in cur:
    print(tb)

('cuisines',)
('evaluation',)
('main_table',)
('ratings',)
('restaurants',)


In [62]:
import pandas as pd
import numpy as np

zomato_dataframe = pd.read_csv('cleaned_zomato.csv')

In [63]:
zomato_dataframe.drop('Unnamed: 0', axis=1, inplace=True)
zomato_dataframe.head(3)


,Restaurant ID,Restaurant Name,Country Code,City,Address,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,Average cost USD,geometry
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...",French,1100,Botswana Pula(P),Yes,No,No,No,3.0,4.8,Dark Green,Excellent,314.0,100.1,POINT (121.027535 14.565443)
1,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...",Japanese,1100,Botswana Pula(P),Yes,No,No,No,3.0,4.8,Dark Green,Excellent,314.0,100.1,POINT (121.027535 14.565443)
2,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...",Desserts,1100,Botswana Pula(P),Yes,No,No,No,3.0,4.8,Dark Green,Excellent,314.0,100.1,POINT (121.027535 14.565443)


In [67]:
subset = zomato_dataframe[zomato_dataframe.columns]
tuples = [tuple(x) for x in subset.to_numpy()]

In [66]:
zomato_dataframe.shape

(19708, 19)

In [65]:
zomato_dataframe.dropna(inplace=True)

In [60]:
sqlFormula = """INSERT INTO main_table 
                    (
                    Restaurant_Name,
                    Country_Code,
                    City,
                    Address,
                    Cuisine,
                    Average_Cost_for_two,
                    Currency,
                    Table_booking,
                    Online_delivery,
                    Now_delivering,
                    Switch_menu,
                    Price_range,
                    Aggregate_rating,
                    Rating_color,
                    Rating_text,
                    Votes,
                    Average_cost_USD,
                    Coordinates
                    ) 
                    VALUES (
                    %s, %s, %s, %s, %s, %s, 
                    %s, %s, %s, %s, %s, %s, 
                    %s, %s, %s, %s, %s, %s, %s
                    );""" #%s placeholders

In [68]:
cur.executemany(sqlFormula, tuples)
connection.commit()

OperationalError: (1136, "Column count doesn't match value count at row 1")

In [24]:
cur.execute("""ALTER TABLE students
ADD COLUMN id serial PRIMARY KEY""")
connection.commit()

OperationalError: (1060, "Duplicate column name 'id'")